In [1]:
"""
Band 1 - Blue	0.45-0.52	30
Band 2 - Green	0.52-0.60	30
Band 3 - Red	0.63-0.69	30
Band 4 - Near Infrared (NIR)	0.77-0.90	30
Band 5 - Shortwave Infrared (SWIR) 1	1.55-1.75	30
Band 6 - Thermal	10.40-12.50	60 (30)
Band 7 - Shortwave Infrared (SWIR) 2	2.09-2.35	30
Band 8 - Panchromatic
"""

BLUE = 0
GREEN = 1
RED = 2
NIR = 3
SWIR_1 = 4
THERMAL = 5
SWIR_2 = 6
PANCH = 7

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
def load_image(file_name):
        image = np.load(file_name, mmap_mode='r')
        image = image['x']

        for c, v in enumerate(image):
            image[c] /= np.max(image[c])

        return image
    
def show_image(image):
    rgb_img = [image[2], image[1], image[0]]
    rgb_img = np.transpose(rgb_img, (1, 2, 0))
    plt.imshow(rgb_img)
    plt.axis('off')
    plt.show()


    
def calc_NDVI(image):
    ndvi = (image[NIR]-image[RED])/(image[NIR]+image[RED])
    return (ndvi, ndvi.mean())

def calc_AWEI(image):
    awei = 4*(image[GREEN]-image[SWIR_1])-0.25*(image[NIR]+2.75*image[SWIR_2])
    return (awei, awei.mean())

def calc_NDWI(image):
    ndwi = (image[GREEN]-image[NIR])/(image[GREEN]+image[NIR])
    return (ndwi, ndwi.mean())

def calc_MNDWI(image):
    mndwi = (image[GREEN]-(image[SWIR_1]+image[SWIR_2]))/(image[NIR]+(image[SWIR_1]+image[SWIR_2]))
    return (mndwi, mndwi.mean())


THRESHOLD = -0.2

def get_waters_preds(image, ndvi):
    x = THRESHOLD
    
    waters_ret = np.ndarray(image.shape)
    waters_ret_sums = [0, 0, 0, 0, 0, 0, 0, 0]
    waters_count = 0
    
    for i_r, r in enumerate(ndvi):
        for i_c, c in enumerate(r):
            if c > x:
                for col in range(8):
                    waters_ret[col][i_r][i_c] = 0
            else:
                waters_count += 1
                for col in range(8):
                    waters_ret[col][i_r][i_c] = image[col][i_r][i_c]
                    waters_ret_sums[col] += image[col][i_r][i_c]
    
    if waters_count != 0:
        waters_ret_sums = [x / waters_count for x in waters_ret_sums]
    
    water_ratio = waters_count/(255*255)
    
#     print(f'{ndvi.shape} - {waters_count} - {water_ratio}')
    return (waters_ret, waters_ret_sums, water_ratio)


class ImageData:
    def __init__(self, file_name):
        self.image = load_image(file_name)
        print('LOADED')
        self.ndvi, self.ndvi_mean = calc_NDVI(self.image)
        self.awei_mean = calc_AWEI(self.image)[1]
        self.ndwi_mean = calc_NDWI(self.image)[1]
        self.mndwi_mean = calc_MNDWI(self.image)[1]



def build_df(name_list: list[str]) -> pd.DataFrame:
    

    data = {
        'index': [],
        'NDVI': [],
        'AWEI': [],
        'NDWI': [],
        'MNDWI': [],
        'BLUE': [],
        'GREEN': [],
        'RED': [],
        'NIR': [],
        'SWIR_1': [],
        'THERMAL': [],
        'SWIR_2': [],
        'PANCH': []
    }

    for index, file_name in enumerate(name_list):
        print(file_name)
        print(index)
#        try:
        img_data = ImageData(file_name)
        waters_image, waters_preds, waters_ratio = get_waters_preds(img_data.image, img_data.ndvi)
        data['index'].append(index)
        data['NDVI'].append(img_data.ndvi_mean)
        data['AWEI'].append(img_data.awei_mean)
        data['NDWI'].append(img_data.ndwi_mean)
        data['MNDWI'].append(img_data.mndwi_mean)
        data['BLUE'].append(waters_preds[BLUE])
        data['GREEN'].append(waters_preds[GREEN])
        data['RED'].append(waters_preds[RED])
        data['NIR'].append(waters_preds[NIR])
        data['SWIR_1'].append(waters_preds[SWIR_1])
        data['THERMAL'].append(waters_preds[THERMAL])
        data['SWIR_2'].append(waters_preds[SWIR_2])
        data['PANCH'].append(waters_preds[PANCH])
#         except:
#             print('ERROR')

    df = pd.DataFrame(data=data)
    df.set_index('index')
    return df

In [ ]:
file_names = [
    './datathon/IA-2015-7-00360473.npz',
    './datathon/IA-2015-7-00360475.npz',
    './datathon/IA-2015-7-00360480.npz',
    './datathon/IA-2015-7-00360481.npz'
]

for name in file_names:
    image = load_image(name)
    ndvi, ndvia_mean = calc_NDVI(image)
    awei, awei_mean = calc_AWEI(image)
    ndwi, ndwi_mean = calc_NDWI(image)
    mndwi, mndwi_mean = calc_MNDWI(image)
    
    
    show_image(image)
    
    waters_image, waters_preds, waters_ratio = get_waters_preds(image, ndvi)
    
    show_image(waters_image)
    print(waters_preds)
    print(waters_ratio)
    
    plt.imshow(ndvi, cmap='hot', interpolation='nearest')
    plt.show()
                
    plt.imshow(awei, cmap='hot', interpolation='nearest')
    plt.show()
    
    plt.imshow(ndwi, cmap='hot', interpolation='nearest')
    plt.show()
    
    plt.imshow(mndwi, cmap='hot', interpolation='nearest')
    plt.show()
    
    print("--------------------------------------------")
    
    

In [ ]:
THRESHOLD = 10 #LOWER 10%

def get_waters(image, ndvi):
    x = np.percentile(ndvi, THRESHOLD)
    print(x)
    
    waters_ret = np.ndarray(image.shape)
    
    for i_r, r in enumerate(ndvi):
        for i_c, c in enumerate(r):
            if c > x:
                for col in range(8):
                    waters_ret[col][i_r][i_c] = 0
            else:
                for col in range(8):
                    waters_ret[col][i_r][i_c] = image[col][i_r][i_c]
    return waters_ret

In [ ]:
df = build_df(file_names)

In [ ]:
df

dhs_train/IA-2015-7-00010027.npz
0
LOADED
dhs_train/IA-2015-7-00020670.npz
1
LOADED
dhs_train/IA-2015-7-00020924.npz
2
LOADED
dhs_train/IA-2015-7-00020926.npz
3
LOADED
dhs_train/IA-2015-7-00030004.npz
4
LOADED
dhs_train/IA-2015-7-00030010.npz
5
LOADED
dhs_train/IA-2015-7-00030020.npz
6
LOADED
dhs_train/IA-2015-7-00030027.npz
7
LOADED
dhs_train/IA-2015-7-00030039.npz
8
LOADED
dhs_train/IA-2015-7-00030048.npz
9
LOADED
dhs_train/IA-2015-7-00030060.npz
10
LOADED
dhs_train/IA-2015-7-00030073.npz
11
LOADED
dhs_train/IA-2015-7-00030096.npz
12
LOADED
dhs_train/IA-2015-7-00030099.npz
13
LOADED
dhs_train/IA-2015-7-00030112.npz
14
LOADED
dhs_train/IA-2015-7-00030196.npz
15
LOADED
dhs_train/IA-2015-7-00030204.npz
16
LOADED
dhs_train/IA-2015-7-00030219.npz
17
LOADED
dhs_train/IA-2015-7-00030233.npz
18
LOADED
dhs_train/IA-2015-7-00030236.npz
19
LOADED
dhs_train/IA-2015-7-00030240.npz
20
LOADED
dhs_train/IA-2015-7-00030266.npz
21
LOADED
dhs_train/IA-2015-7-00030269.npz
22
LOADED
dhs_train/IA-2015-7-0

dhs_train/IA-2015-7-00040545.npz
189
LOADED
dhs_train/IA-2015-7-00040549.npz
190
LOADED
dhs_train/IA-2015-7-00040553.npz
191
LOADED
dhs_train/IA-2015-7-00040554.npz
192
LOADED
dhs_train/IA-2015-7-00040556.npz
193
LOADED
dhs_train/IA-2015-7-00040557.npz
194
LOADED
dhs_train/IA-2015-7-00040562.npz
195
LOADED
dhs_train/IA-2015-7-00040568.npz
196
LOADED
dhs_train/IA-2015-7-00040571.npz
197
LOADED
dhs_train/IA-2015-7-00040577.npz
198
LOADED
dhs_train/IA-2015-7-00040585.npz
199
LOADED
dhs_train/IA-2015-7-00040587.npz
200
LOADED
dhs_train/IA-2015-7-00040590.npz
201
LOADED
dhs_train/IA-2015-7-00040591.npz
202
LOADED
dhs_train/IA-2015-7-00040601.npz
203
LOADED
dhs_train/IA-2015-7-00040603.npz
204
LOADED
dhs_train/IA-2015-7-00040607.npz
205
LOADED
dhs_train/IA-2015-7-00040610.npz
206
LOADED
dhs_train/IA-2015-7-00040614.npz
207
LOADED
dhs_train/IA-2015-7-00040617.npz
208
LOADED
dhs_train/IA-2015-7-00040619.npz
209
LOADED
dhs_train/IA-2015-7-00040623.npz
210
LOADED
dhs_train/IA-2015-7-00040627.npz

dhs_train/IA-2015-7-00050076.npz
376
LOADED
dhs_train/IA-2015-7-00050081.npz
377
LOADED
dhs_train/IA-2015-7-00050082.npz
378
LOADED
dhs_train/IA-2015-7-00050084.npz
379
LOADED
dhs_train/IA-2015-7-00050085.npz
380
LOADED
dhs_train/IA-2015-7-00050091.npz
381
LOADED
dhs_train/IA-2015-7-00050092.npz
382
LOADED
dhs_train/IA-2015-7-00050101.npz
383
LOADED
dhs_train/IA-2015-7-00050102.npz
384
LOADED
dhs_train/IA-2015-7-00050103.npz
385
LOADED
dhs_train/IA-2015-7-00050106.npz
386
LOADED
dhs_train/IA-2015-7-00050108.npz
387
LOADED
dhs_train/IA-2015-7-00050109.npz
388
LOADED
dhs_train/IA-2015-7-00050115.npz
389
LOADED
dhs_train/IA-2015-7-00050118.npz
390
LOADED
dhs_train/IA-2015-7-00050119.npz
391
LOADED
dhs_train/IA-2015-7-00050123.npz
392
LOADED
dhs_train/IA-2015-7-00050124.npz
393
LOADED
dhs_train/IA-2015-7-00050125.npz
394
LOADED
dhs_train/IA-2015-7-00050126.npz
395
LOADED
dhs_train/IA-2015-7-00050129.npz
396
LOADED
dhs_train/IA-2015-7-00050130.npz
397
LOADED
dhs_train/IA-2015-7-00050134.npz

dhs_train/IA-2015-7-00050547.npz
563
LOADED
dhs_train/IA-2015-7-00050549.npz
564
LOADED
dhs_train/IA-2015-7-00050550.npz
565
LOADED
dhs_train/IA-2015-7-00050555.npz
566
LOADED
dhs_train/IA-2015-7-00050562.npz
567
LOADED
dhs_train/IA-2015-7-00050563.npz
568
LOADED
dhs_train/IA-2015-7-00050564.npz
569
LOADED
dhs_train/IA-2015-7-00050565.npz
570
LOADED
dhs_train/IA-2015-7-00050568.npz
571
LOADED
dhs_train/IA-2015-7-00050569.npz
572
LOADED
dhs_train/IA-2015-7-00050570.npz
573
LOADED
dhs_train/IA-2015-7-00050575.npz
574
LOADED
dhs_train/IA-2015-7-00050576.npz
575
LOADED
dhs_train/IA-2015-7-00050579.npz
576
LOADED
dhs_train/IA-2015-7-00050581.npz
577
LOADED
dhs_train/IA-2015-7-00050586.npz
578
LOADED
dhs_train/IA-2015-7-00050588.npz
579
LOADED
dhs_train/IA-2015-7-00050590.npz
580
LOADED
dhs_train/IA-2015-7-00050591.npz
581
LOADED
dhs_train/IA-2015-7-00050597.npz
582
LOADED
dhs_train/IA-2015-7-00050600.npz
583
LOADED
dhs_train/IA-2015-7-00050601.npz
584
LOADED
dhs_train/IA-2015-7-00050609.npz

KeyboardInterrupt: 

In [12]:
no_chunks = 5

chunks = np.array_split(df, no_chunks)

for i in range(no_chunks):
    chunks[i].to_csv(f'train_chunk_{i}.csv')


In [13]:
file_name = 'train_chunk_0.csv'
df = pd.read_csv (file_name)
added_df = build_df(list(df['path']))
df.join(added_df)
df.to_csv('train_chunk_0_v2.csv')

dhs_train/IA-2015-7-00010027.npz
0
LOADED
dhs_train/IA-2015-7-00020670.npz
1
LOADED
dhs_train/IA-2015-7-00020924.npz
2
LOADED
dhs_train/IA-2015-7-00020926.npz
3
LOADED
dhs_train/IA-2015-7-00030004.npz
4
LOADED
dhs_train/IA-2015-7-00030010.npz
5
LOADED
dhs_train/IA-2015-7-00030020.npz
6
LOADED
dhs_train/IA-2015-7-00030027.npz
7
LOADED
dhs_train/IA-2015-7-00030039.npz
8
LOADED
dhs_train/IA-2015-7-00030048.npz
9
LOADED
dhs_train/IA-2015-7-00030060.npz
10
LOADED
dhs_train/IA-2015-7-00030073.npz
11
LOADED
dhs_train/IA-2015-7-00030096.npz
12
LOADED
dhs_train/IA-2015-7-00030099.npz
13
LOADED
dhs_train/IA-2015-7-00030112.npz
14
LOADED
dhs_train/IA-2015-7-00030196.npz
15
LOADED
dhs_train/IA-2015-7-00030204.npz
16
LOADED
dhs_train/IA-2015-7-00030219.npz
17
LOADED
dhs_train/IA-2015-7-00030233.npz
18
LOADED
dhs_train/IA-2015-7-00030236.npz
19
LOADED
dhs_train/IA-2015-7-00030240.npz
20
LOADED
dhs_train/IA-2015-7-00030266.npz
21
LOADED
dhs_train/IA-2015-7-00030269.npz
22
LOADED
dhs_train/IA-2015-7-0

dhs_train/IA-2015-7-00040545.npz
189
LOADED
dhs_train/IA-2015-7-00040549.npz
190
LOADED
dhs_train/IA-2015-7-00040553.npz
191
LOADED
dhs_train/IA-2015-7-00040554.npz
192
LOADED
dhs_train/IA-2015-7-00040556.npz
193
LOADED
dhs_train/IA-2015-7-00040557.npz
194
LOADED
dhs_train/IA-2015-7-00040562.npz
195
LOADED
dhs_train/IA-2015-7-00040568.npz
196
LOADED
dhs_train/IA-2015-7-00040571.npz
197
LOADED
dhs_train/IA-2015-7-00040577.npz
198
LOADED
dhs_train/IA-2015-7-00040585.npz
199
LOADED
dhs_train/IA-2015-7-00040587.npz
200
LOADED
dhs_train/IA-2015-7-00040590.npz
201
LOADED
dhs_train/IA-2015-7-00040591.npz
202
LOADED
dhs_train/IA-2015-7-00040601.npz
203
LOADED
dhs_train/IA-2015-7-00040603.npz
204
LOADED
dhs_train/IA-2015-7-00040607.npz
205
LOADED
dhs_train/IA-2015-7-00040610.npz
206
LOADED
dhs_train/IA-2015-7-00040614.npz
207
LOADED
dhs_train/IA-2015-7-00040617.npz
208
LOADED
dhs_train/IA-2015-7-00040619.npz
209
LOADED
dhs_train/IA-2015-7-00040623.npz
210
LOADED
dhs_train/IA-2015-7-00040627.npz

dhs_train/IA-2015-7-00050076.npz
376
LOADED
dhs_train/IA-2015-7-00050081.npz
377
LOADED
dhs_train/IA-2015-7-00050082.npz
378
LOADED
dhs_train/IA-2015-7-00050084.npz
379
LOADED
dhs_train/IA-2015-7-00050085.npz
380
LOADED
dhs_train/IA-2015-7-00050091.npz
381
LOADED
dhs_train/IA-2015-7-00050092.npz
382
LOADED
dhs_train/IA-2015-7-00050101.npz
383
LOADED
dhs_train/IA-2015-7-00050102.npz
384
LOADED
dhs_train/IA-2015-7-00050103.npz
385
LOADED
dhs_train/IA-2015-7-00050106.npz
386
LOADED
dhs_train/IA-2015-7-00050108.npz
387
LOADED
dhs_train/IA-2015-7-00050109.npz
388
LOADED
dhs_train/IA-2015-7-00050115.npz
389
LOADED
dhs_train/IA-2015-7-00050118.npz
390
LOADED
dhs_train/IA-2015-7-00050119.npz
391
LOADED
dhs_train/IA-2015-7-00050123.npz
392
LOADED
dhs_train/IA-2015-7-00050124.npz
393
LOADED
dhs_train/IA-2015-7-00050125.npz
394
LOADED
dhs_train/IA-2015-7-00050126.npz
395
LOADED
dhs_train/IA-2015-7-00050129.npz
396
LOADED
dhs_train/IA-2015-7-00050130.npz
397
LOADED
dhs_train/IA-2015-7-00050134.npz

dhs_train/IA-2015-7-00050547.npz
563
LOADED
dhs_train/IA-2015-7-00050549.npz
564
LOADED
dhs_train/IA-2015-7-00050550.npz
565
LOADED
dhs_train/IA-2015-7-00050555.npz
566
LOADED
dhs_train/IA-2015-7-00050562.npz
567
LOADED
dhs_train/IA-2015-7-00050563.npz
568
LOADED
dhs_train/IA-2015-7-00050564.npz
569
LOADED
dhs_train/IA-2015-7-00050565.npz
570
LOADED
dhs_train/IA-2015-7-00050568.npz
571
LOADED
dhs_train/IA-2015-7-00050569.npz
572
LOADED
dhs_train/IA-2015-7-00050570.npz
573
LOADED
dhs_train/IA-2015-7-00050575.npz
574
LOADED
dhs_train/IA-2015-7-00050576.npz
575
LOADED
dhs_train/IA-2015-7-00050579.npz
576
LOADED
dhs_train/IA-2015-7-00050581.npz
577
LOADED
dhs_train/IA-2015-7-00050586.npz
578
LOADED
dhs_train/IA-2015-7-00050588.npz
579
LOADED
dhs_train/IA-2015-7-00050590.npz
580
LOADED
dhs_train/IA-2015-7-00050591.npz
581
LOADED
dhs_train/IA-2015-7-00050597.npz
582
LOADED
dhs_train/IA-2015-7-00050600.npz
583
LOADED
dhs_train/IA-2015-7-00050601.npz
584
LOADED
dhs_train/IA-2015-7-00050609.npz

dhs_train/IA-2015-7-00051035.npz
750
LOADED
dhs_train/IA-2015-7-00051036.npz
751
LOADED
dhs_train/IA-2015-7-00051037.npz
752
LOADED
dhs_train/IA-2015-7-00051043.npz
753
LOADED
dhs_train/IA-2015-7-00051044.npz
754
LOADED
dhs_train/IA-2015-7-00051046.npz
755
LOADED
dhs_train/IA-2015-7-00051048.npz
756
LOADED
dhs_train/IA-2015-7-00051051.npz
757
LOADED
dhs_train/IA-2015-7-00051052.npz
758
LOADED
dhs_train/IA-2015-7-00051054.npz
759
LOADED
dhs_train/IA-2015-7-00051056.npz
760
LOADED
dhs_train/IA-2015-7-00051063.npz
761
LOADED
dhs_train/IA-2015-7-00051064.npz
762
LOADED
dhs_train/IA-2015-7-00051065.npz
763
LOADED
dhs_train/IA-2015-7-00051066.npz
764
LOADED
dhs_train/IA-2015-7-00051068.npz
765
LOADED
dhs_train/IA-2015-7-00051069.npz
766
LOADED
dhs_train/IA-2015-7-00051070.npz
767
LOADED
dhs_train/IA-2015-7-00051071.npz
768
LOADED
dhs_train/IA-2015-7-00051074.npz
769
LOADED
dhs_train/IA-2015-7-00051079.npz
770
LOADED
dhs_train/IA-2015-7-00051081.npz
771
LOADED
dhs_train/IA-2015-7-00051082.npz

...

dhs_train/IA-2015-7-00260222.npz
2005
LOADED
dhs_train/IA-2015-7-00260228.npz
2006
LOADED
dhs_train/IA-2015-7-00260237.npz
2007
LOADED
dhs_train/IA-2015-7-00260265.npz
2008
LOADED
dhs_train/IA-2015-7-00260268.npz
2009
LOADED
dhs_train/IA-2015-7-00260269.npz
2010
LOADED
dhs_train/IA-2015-7-00260285.npz
2011
LOADED
dhs_train/IA-2015-7-00260287.npz
2012
LOADED
dhs_train/IA-2015-7-00260288.npz
2013
LOADED
dhs_train/IA-2015-7-00260290.npz
2014
LOADED
dhs_train/IA-2015-7-00260292.npz
2015
LOADED
dhs_train/IA-2015-7-00260301.npz
2016
LOADED
dhs_train/IA-2015-7-00260307.npz
2017
LOADED
dhs_train/IA-2015-7-00260317.npz
2018
LOADED
dhs_train/IA-2015-7-00260328.npz
2019
LOADED
dhs_train/IA-2015-7-00260340.npz
2020
LOADED
dhs_train/IA-2015-7-00260352.npz
2021
LOADED
dhs_train/IA-2015-7-00260356.npz
2022
LOADED
dhs_train/IA-2015-7-00260358.npz
2023
LOADED
dhs_train/IA-2015-7-00260361.npz
2024
LOADED
dhs_train/IA-2015-7-00260363.npz
2025
LOADED
dhs_train/IA-2015-7-00260365.npz
2026
LOADED
dhs_train/

dhs_train/IA-2015-7-00261363.npz
2188
LOADED
dhs_train/IA-2015-7-00261367.npz
2189
LOADED
dhs_train/IA-2015-7-00261386.npz
2190
LOADED
dhs_train/IA-2015-7-00261396.npz
2191
LOADED
dhs_train/IA-2015-7-00261400.npz
2192
LOADED
dhs_train/IA-2015-7-00261424.npz
2193
LOADED
dhs_train/IA-2015-7-00261425.npz
2194
LOADED
dhs_train/IA-2015-7-00261429.npz
2195
LOADED
dhs_train/IA-2015-7-00261433.npz
2196
LOADED
dhs_train/IA-2015-7-00261445.npz
2197
LOADED
dhs_train/IA-2015-7-00261448.npz
2198
LOADED
dhs_train/IA-2015-7-00261450.npz
2199
LOADED
dhs_train/IA-2015-7-00261451.npz
2200
LOADED
dhs_train/IA-2015-7-00261462.npz
2201
LOADED
dhs_train/IA-2015-7-00261463.npz
2202
LOADED
dhs_train/IA-2015-7-00261464.npz
2203
LOADED
dhs_train/IA-2015-7-00261469.npz
2204
LOADED
dhs_train/IA-2015-7-00261479.npz
2205
LOADED
dhs_train/IA-2015-7-00261481.npz
2206
LOADED
dhs_train/IA-2015-7-00261492.npz
2207
LOADED
dhs_train/IA-2015-7-00261494.npz
2208
LOADED
dhs_train/IA-2015-7-00261496.npz
2209
LOADED
dhs_train/

dhs_train/IA-2015-7-00330814.npz
2371
LOADED
dhs_train/IA-2015-7-00330815.npz
2372
LOADED
dhs_train/IA-2015-7-00330818.npz
2373
LOADED
dhs_train/IA-2015-7-00330829.npz
2374
LOADED
dhs_train/IA-2015-7-00330841.npz
2375
LOADED
dhs_train/IA-2015-7-00330848.npz
2376
LOADED
dhs_train/IA-2015-7-00330849.npz
2377
LOADED
dhs_train/IA-2015-7-00330851.npz
2378
LOADED
dhs_train/IA-2015-7-00330855.npz
2379
LOADED
dhs_train/IA-2015-7-00330861.npz
2380
LOADED
dhs_train/IA-2015-7-00330862.npz
2381
LOADED
dhs_train/IA-2015-7-00330869.npz
2382
LOADED
dhs_train/IA-2015-7-00330872.npz
2383
LOADED
dhs_train/IA-2015-7-00330874.npz
2384
LOADED
dhs_train/IA-2015-7-00330876.npz
2385
LOADED
dhs_train/IA-2015-7-00330882.npz
2386
LOADED
dhs_train/IA-2015-7-00330883.npz
2387
LOADED
dhs_train/IA-2015-7-00330898.npz
2388
LOADED
dhs_train/IA-2015-7-00330911.npz
2389
LOADED
dhs_train/IA-2015-7-00330916.npz
2390
LOADED
dhs_train/IA-2015-7-00330926.npz
2391
LOADED
dhs_train/IA-2015-7-00330927.npz
2392
LOADED
dhs_train/

dhs_train/IA-2015-7-00331909.npz
2554
LOADED
dhs_train/IA-2015-7-00331910.npz
2555
LOADED
dhs_train/IA-2015-7-00331919.npz
2556
LOADED
dhs_train/IA-2015-7-00331921.npz
2557
LOADED
dhs_train/IA-2015-7-00331929.npz
2558
LOADED
dhs_train/IA-2015-7-00331933.npz
2559
LOADED
dhs_train/IA-2015-7-00331940.npz
2560
LOADED
dhs_train/IA-2015-7-00331942.npz
2561
LOADED
dhs_train/IA-2015-7-00331944.npz
2562
LOADED
dhs_train/IA-2015-7-00331952.npz
2563
LOADED
dhs_train/IA-2015-7-00331954.npz
2564
LOADED
dhs_train/IA-2015-7-00331959.npz
2565
LOADED
dhs_train/IA-2015-7-00331967.npz
2566
LOADED
dhs_train/IA-2015-7-00331972.npz
2567
LOADED
dhs_train/IA-2015-7-00331973.npz
2568
LOADED
dhs_train/IA-2015-7-00331979.npz
2569
LOADED
dhs_train/IA-2015-7-00332002.npz
2570
LOADED
dhs_train/IA-2015-7-00332013.npz
2571
LOADED
dhs_train/IA-2015-7-00332026.npz
2572
LOADED
dhs_train/IA-2015-7-00332028.npz
2573
LOADED
dhs_train/IA-2015-7-00332044.npz
2574
LOADED
dhs_train/IA-2015-7-00332055.npz
2575
LOADED
dhs_train/

dhs_train/IA-2015-7-00332945.npz
2737
LOADED
dhs_train/IA-2015-7-00332947.npz
2738
LOADED
dhs_train/IA-2015-7-00332948.npz
2739
LOADED
dhs_train/IA-2015-7-00332956.npz
2740
LOADED
dhs_train/IA-2015-7-00332959.npz
2741
LOADED
dhs_train/IA-2015-7-00332966.npz
2742
LOADED
dhs_train/IA-2015-7-00332970.npz
2743
LOADED
dhs_train/IA-2015-7-00332972.npz
2744
LOADED
dhs_train/IA-2015-7-00332983.npz
2745
LOADED
dhs_train/IA-2015-7-00332991.npz
2746
LOADED
dhs_train/IA-2015-7-00332993.npz
2747
LOADED
dhs_train/IA-2015-7-00332994.npz
2748
LOADED
dhs_train/IA-2015-7-00333009.npz
2749
LOADED
dhs_train/IA-2015-7-00333010.npz
2750
LOADED
dhs_train/IA-2015-7-00333012.npz
2751
LOADED
dhs_train/IA-2015-7-00333019.npz
2752
LOADED
dhs_train/IA-2015-7-00333020.npz
2753
LOADED
dhs_train/IA-2015-7-00333034.npz
2754
LOADED
dhs_train/IA-2015-7-00333048.npz
2755
LOADED
dhs_train/IA-2015-7-00333056.npz
2756
LOADED
dhs_train/IA-2015-7-00333058.npz
2757
LOADED
dhs_train/IA-2015-7-00333062.npz
2758
LOADED
dhs_train/

In [14]:
df

,Unnamed: 0.1,Unnamed: 0,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,cluster_id,adm1dhs,urban,path
0,0,1,IA-2015-7-00010027,IA,2015,12.472236,92.883517,22,-2.836691,22,1.136364,10027,1,R,dhs_train/IA-2015-7-00010027.npz
1,1,2,IA-2015-7-00020670,IA,2015,16.815604,81.329535,21,0.732851,21,2.523810,20670,2,R,dhs_train/IA-2015-7-00020670.npz
2,2,3,IA-2015-7-00020924,IA,2015,16.180453,80.525736,21,1.207174,21,3.000000,20924,2,R,dhs_train/IA-2015-7-00020924.npz
3,3,4,IA-2015-7-00020926,IA,2015,17.514527,82.077392,21,-1.034582,21,3.000000,20926,2,R,dhs_train/IA-2015-7-00020926.npz
4,4,5,IA-2015-7-00030004,IA,2015,27.524358,96.010830,19,-0.106301,19,3.000000,30004,3,R,dhs_train/IA-2015-7-00030004.npz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,2803,2804,IA-2015-7-00333303,IA,2015,28.983935,78.974596,23,1.164709,23,3.000000,333303,33,R,dhs_train/IA-2015-7-00333303.npz
2804,2804,2805,IA-2015-7-00333308,IA,2015,28.450035,80.302918,21,-1.885487,21,3.000000,333308,33,R,dhs_train/IA-2015-7-00333308.npz
2805,2805,2806,IA-2015-7-00333312,IA,2015,26.947539,82.929313,20,-2.487550,20,3.000000,333312,33,R,dhs_train/IA-2015-7-00333312.npz
2806,2806,2807,IA-2015-7-00333318,IA,2015,27.111957,78.935074,22,-1.140653,22,3.000000,333318,33,R,dhs_train/IA-2015-7-00333318.npz


In [15]:
df.join(added_df)

,Unnamed: 0.1,Unnamed: 0,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,n_water,...,NDWI,MNDWI,BLUE,GREEN,RED,NIR,SWIR_1,THERMAL,SWIR_2,PANCH
0,0,1,IA-2015-7-00010027,IA,2015,12.472236,92.883517,22,-2.836691,22,...,0.143159,-0.336063,0.218351,0.334681,0.184333,0.044674,0.038004,0.980060,0.130060,0.006853
1,1,2,IA-2015-7-00020670,IA,2015,16.815604,81.329535,21,0.732851,21,...,0.156040,-0.299680,0.614745,0.632684,0.537778,0.322646,0.216603,0.987814,0.626100,0.234687
2,2,3,IA-2015-7-00020924,IA,2015,16.180453,80.525736,21,1.207174,21,...,-0.091609,-0.447495,0.258158,0.288624,0.193837,0.095857,0.063848,0.980387,0.256224,0.315863
3,3,4,IA-2015-7-00020926,IA,2015,17.514527,82.077392,21,-1.034582,21,...,0.058564,-0.325924,0.581038,0.637503,0.490663,0.269533,0.210301,0.988735,0.452360,0.249139
4,4,5,IA-2015-7-00030004,IA,2015,27.524358,96.010830,19,-0.106301,19,...,-0.135953,-0.358507,0.525010,0.540212,0.473602,0.231442,0.202817,0.984314,0.277958,0.001738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,2803,2804,IA-2015-7-00333303,IA,2015,28.983935,78.974596,23,1.164709,23,...,0.026006,-0.372082,0.442066,0.477818,0.335263,0.181920,0.132787,0.986121,0.443936,0.209350
2804,2804,2805,IA-2015-7-00333308,IA,2015,28.450035,80.302918,21,-1.885487,21,...,0.117325,-0.358059,0.630089,0.637787,0.465384,0.224468,0.158018,0.988692,0.511168,0.408831
2805,2805,2806,IA-2015-7-00333312,IA,2015,26.947539,82.929313,20,-2.487550,20,...,0.037404,-0.366460,0.657431,0.667453,0.533018,0.250700,0.183553,0.989202,0.532445,0.624525
2806,2806,2807,IA-2015-7-00333318,IA,2015,27.111957,78.935074,22,-1.140653,22,...,-0.042606,-0.415184,0.423123,0.473474,0.396342,0.228039,0.180650,0.984128,0.424377,0.298447


In [17]:
added_df.join(df).to_csv('train_chunk_0_v2.csv')

In [18]:
new_ = pd.read_csv ('train_chunk_0_v2.csv')

In [19]:
new_

,Unnamed: 0.2,index,NDVI,AWEI,NDWI,MNDWI,BLUE,GREEN,RED,NIR,...,lat,lon,n_asset,asset_index,n_water,water_index,cluster_id,adm1dhs,urban,path
0,0,0,0.150913,0.137764,0.143159,-0.336063,0.218351,0.334681,0.184333,0.044674,...,12.472236,92.883517,22,-2.836691,22,1.136364,10027,1,R,dhs_train/IA-2015-7-00010027.npz
1,1,1,-0.061623,0.519389,0.156040,-0.299680,0.614745,0.632684,0.537778,0.322646,...,16.815604,81.329535,21,0.732851,21,2.523810,20670,2,R,dhs_train/IA-2015-7-00020670.npz
2,2,2,0.175093,-0.350465,-0.091609,-0.447495,0.258158,0.288624,0.193837,0.095857,...,16.180453,80.525736,21,1.207174,21,3.000000,20924,2,R,dhs_train/IA-2015-7-00020924.npz
3,3,3,0.097361,0.192039,0.058564,-0.325924,0.581038,0.637503,0.490663,0.269533,...,17.514527,82.077392,21,-1.034582,21,3.000000,20926,2,R,dhs_train/IA-2015-7-00020926.npz
4,4,4,0.209335,-0.463782,-0.135953,-0.358507,0.525010,0.540212,0.473602,0.231442,...,27.524358,96.010830,19,-0.106301,19,3.000000,30004,3,R,dhs_train/IA-2015-7-00030004.npz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,2803,2803,0.090251,-0.059005,0.026006,-0.372082,0.442066,0.477818,0.335263,0.181920,...,28.983935,78.974596,23,1.164709,23,3.000000,333303,33,R,dhs_train/IA-2015-7-00333303.npz
2804,2804,2804,0.096958,0.382621,0.117325,-0.358059,0.630089,0.637787,0.465384,0.224468,...,28.450035,80.302918,21,-1.885487,21,3.000000,333308,33,R,dhs_train/IA-2015-7-00333308.npz
2805,2805,2805,0.056700,-0.045293,0.037404,-0.366460,0.657431,0.667453,0.533018,0.250700,...,26.947539,82.929313,20,-2.487550,20,3.000000,333312,33,R,dhs_train/IA-2015-7-00333312.npz
2806,2806,2806,0.106174,-0.402866,-0.042606,-0.415184,0.423123,0.473474,0.396342,0.228039,...,27.111957,78.935074,22,-1.140653,22,3.000000,333318,33,R,dhs_train/IA-2015-7-00333318.npz


In [111]:
file_name = 'water_test.csv'
df = pd.read_csv (file_name)
added_df = build_df(list(df['path']))
df.join(added_df)
new_v = added_df.join(df)
new_v.to_csv('water_test.csv')

dhs_train/IA-2015-7-00010021.npz
0
LOADED
dhs_train/IA-2015-7-00010048.npz
1
LOADED
dhs_train/IA-2015-7-00010073.npz
2
LOADED
dhs_train/IA-2015-7-00010082.npz
3
LOADED
dhs_train/IA-2015-7-00010115.npz
4
LOADED
dhs_train/IA-2015-7-00010128.npz
5
LOADED
dhs_train/IA-2015-7-00010132.npz
6
LOADED
dhs_train/IA-2015-7-00010144.npz
7
LOADED
dhs_train/IA-2015-7-00010179.npz
8
LOADED
dhs_train/IA-2015-7-00010190.npz
9
LOADED
dhs_train/IA-2015-7-00020004.npz
10
LOADED
dhs_train/IA-2015-7-00020031.npz
11
LOADED
dhs_train/IA-2015-7-00020035.npz
12
LOADED
dhs_train/IA-2015-7-00020038.npz
13
LOADED
dhs_train/IA-2015-7-00020060.npz
14
LOADED
dhs_train/IA-2015-7-00020073.npz
15
LOADED
dhs_train/IA-2015-7-00020079.npz
16
LOADED
dhs_train/IA-2015-7-00020082.npz
17
LOADED
dhs_train/IA-2015-7-00020102.npz
18
LOADED
dhs_train/IA-2015-7-00020127.npz
19
LOADED
dhs_train/IA-2015-7-00020133.npz
20
LOADED
dhs_train/IA-2015-7-00020157.npz
21
LOADED
dhs_train/IA-2015-7-00020170.npz
22
LOADED
dhs_train/IA-2015-7-0

dhs_train/IA-2015-7-00030513.npz
189
LOADED
dhs_train/IA-2015-7-00030519.npz
190
LOADED
dhs_train/IA-2015-7-00030520.npz
191
LOADED
dhs_train/IA-2015-7-00030532.npz
192
LOADED
dhs_train/IA-2015-7-00030535.npz
193
LOADED
dhs_train/IA-2015-7-00030536.npz
194
LOADED
dhs_train/IA-2015-7-00030540.npz
195
LOADED
dhs_train/IA-2015-7-00030544.npz
196
LOADED
dhs_train/IA-2015-7-00030547.npz
197
LOADED
dhs_train/IA-2015-7-00030558.npz
198
LOADED
dhs_train/IA-2015-7-00030567.npz
199
LOADED
dhs_train/IA-2015-7-00030580.npz
200
LOADED
dhs_train/IA-2015-7-00030602.npz
201
LOADED
dhs_train/IA-2015-7-00030616.npz
202
LOADED
dhs_train/IA-2015-7-00030623.npz
203
LOADED
dhs_train/IA-2015-7-00030629.npz
204
LOADED
dhs_train/IA-2015-7-00030632.npz
205
LOADED
dhs_train/IA-2015-7-00030653.npz
206
LOADED
dhs_train/IA-2015-7-00030663.npz
207
LOADED
dhs_train/IA-2015-7-00030677.npz
208
LOADED
dhs_train/IA-2015-7-00030679.npz
209
LOADED
dhs_train/IA-2015-7-00030694.npz
210
LOADED
dhs_train/IA-2015-7-00030695.npz

dhs_train/IA-2015-7-00040914.npz
376
LOADED
dhs_train/IA-2015-7-00040917.npz
377
LOADED
dhs_train/IA-2015-7-00040923.npz
378
LOADED
dhs_train/IA-2015-7-00040926.npz
379
LOADED
dhs_train/IA-2015-7-00040943.npz
380
LOADED
dhs_train/IA-2015-7-00040949.npz
381
LOADED
dhs_train/IA-2015-7-00040963.npz
382
LOADED
dhs_train/IA-2015-7-00040980.npz
383
LOADED
dhs_train/IA-2015-7-00040991.npz
384
LOADED
dhs_train/IA-2015-7-00041013.npz
385
LOADED
dhs_train/IA-2015-7-00041030.npz
386
LOADED
dhs_train/IA-2015-7-00041033.npz
387
LOADED
dhs_train/IA-2015-7-00041035.npz
388
LOADED
dhs_train/IA-2015-7-00041039.npz
389
LOADED
dhs_train/IA-2015-7-00041041.npz
390
LOADED
dhs_train/IA-2015-7-00041056.npz
391
LOADED
dhs_train/IA-2015-7-00041063.npz
392
LOADED
dhs_train/IA-2015-7-00041064.npz
393
LOADED
dhs_train/IA-2015-7-00041083.npz
394
LOADED
dhs_train/IA-2015-7-00041084.npz
395
LOADED
dhs_train/IA-2015-7-00041088.npz
396
LOADED
dhs_train/IA-2015-7-00041103.npz
397
LOADED
dhs_train/IA-2015-7-00041104.npz

dhs_train/IA-2015-7-00050979.npz
563
LOADED
dhs_train/IA-2015-7-00050980.npz
564
LOADED
dhs_train/IA-2015-7-00050982.npz
565
LOADED
dhs_train/IA-2015-7-00050983.npz
566
LOADED
dhs_train/IA-2015-7-00050987.npz
567
LOADED
dhs_train/IA-2015-7-00050993.npz
568
LOADED
dhs_train/IA-2015-7-00050996.npz
569
LOADED
dhs_train/IA-2015-7-00051008.npz
570
LOADED
dhs_train/IA-2015-7-00051020.npz
571
LOADED
dhs_train/IA-2015-7-00051021.npz
572
LOADED
dhs_train/IA-2015-7-00051022.npz
573
LOADED
dhs_train/IA-2015-7-00051039.npz
574
LOADED
dhs_train/IA-2015-7-00051040.npz
575
LOADED
dhs_train/IA-2015-7-00051041.npz
576
LOADED
dhs_train/IA-2015-7-00051042.npz
577
LOADED
dhs_train/IA-2015-7-00051045.npz
578
LOADED
dhs_train/IA-2015-7-00051047.npz
579
LOADED
dhs_train/IA-2015-7-00051049.npz
580
LOADED
dhs_train/IA-2015-7-00051059.npz
581
LOADED
dhs_train/IA-2015-7-00051080.npz
582
LOADED
dhs_train/IA-2015-7-00051086.npz
583
LOADED
dhs_train/IA-2015-7-00051095.npz
584
LOADED
dhs_train/IA-2015-7-00051097.npz

dhs_train/IA-2015-7-00070401.npz
750
LOADED
dhs_train/IA-2015-7-00070423.npz
751
LOADED
dhs_train/IA-2015-7-00070452.npz
752
LOADED
dhs_train/IA-2015-7-00070458.npz
753
LOADED
dhs_train/IA-2015-7-00070469.npz
754
LOADED
dhs_train/IA-2015-7-00070470.npz
755
LOADED
dhs_train/IA-2015-7-00070475.npz
756
LOADED
dhs_train/IA-2015-7-00070480.npz
757
LOADED
dhs_train/IA-2015-7-00070481.npz
758
LOADED
dhs_train/IA-2015-7-00070485.npz
759
LOADED
dhs_train/IA-2015-7-00070494.npz
760
LOADED
dhs_train/IA-2015-7-00070513.npz
761
LOADED
dhs_train/IA-2015-7-00070515.npz
762
LOADED
dhs_train/IA-2015-7-00070518.npz
763
LOADED
dhs_train/IA-2015-7-00070519.npz
764
LOADED
dhs_train/IA-2015-7-00070522.npz
765
LOADED
dhs_train/IA-2015-7-00070530.npz
766
LOADED
dhs_train/IA-2015-7-00070535.npz
767
LOADED
dhs_train/IA-2015-7-00070537.npz
768
LOADED
dhs_train/IA-2015-7-00070541.npz
769
LOADED
dhs_train/IA-2015-7-00070544.npz
770
LOADED
dhs_train/IA-2015-7-00070552.npz
771
LOADED
dhs_train/IA-2015-7-00070559.npz

...

dhs_train/IA-2015-7-00331218.npz
3818
LOADED
dhs_train/IA-2015-7-00331233.npz
3819
LOADED
dhs_train/IA-2015-7-00331275.npz
3820
LOADED
dhs_train/IA-2015-7-00331286.npz
3821
LOADED
dhs_train/IA-2015-7-00331291.npz
3822
LOADED
dhs_train/IA-2015-7-00331299.npz
3823
LOADED
dhs_train/IA-2015-7-00331300.npz
3824
LOADED
dhs_train/IA-2015-7-00331305.npz
3825
LOADED
dhs_train/IA-2015-7-00331310.npz
3826
LOADED
dhs_train/IA-2015-7-00331313.npz
3827
LOADED
dhs_train/IA-2015-7-00331317.npz
3828
LOADED
dhs_train/IA-2015-7-00331319.npz
3829
LOADED
dhs_train/IA-2015-7-00331322.npz
3830
LOADED
dhs_train/IA-2015-7-00331327.npz
3831
LOADED
dhs_train/IA-2015-7-00331333.npz
3832
LOADED
dhs_train/IA-2015-7-00331334.npz
3833
LOADED
dhs_train/IA-2015-7-00331338.npz
3834
LOADED
dhs_train/IA-2015-7-00331341.npz
3835
LOADED
dhs_train/IA-2015-7-00331342.npz
3836
LOADED
dhs_train/IA-2015-7-00331347.npz
3837
LOADED
dhs_train/IA-2015-7-00331351.npz
3838
LOADED
dhs_train/IA-2015-7-00331356.npz
3839
LOADED
dhs_train/

dhs_train/IA-2015-7-00332835.npz
4001
LOADED
dhs_train/IA-2015-7-00332837.npz
4002
LOADED
dhs_train/IA-2015-7-00332861.npz
4003
LOADED
dhs_train/IA-2015-7-00332889.npz
4004
LOADED
dhs_train/IA-2015-7-00332895.npz
4005
LOADED
dhs_train/IA-2015-7-00332916.npz
4006
LOADED
dhs_train/IA-2015-7-00332926.npz
4007
LOADED
dhs_train/IA-2015-7-00332928.npz
4008
LOADED
dhs_train/IA-2015-7-00332946.npz
4009
LOADED
dhs_train/IA-2015-7-00332950.npz
4010
LOADED
dhs_train/IA-2015-7-00332953.npz
4011
LOADED
dhs_train/IA-2015-7-00332954.npz
4012
LOADED
dhs_train/IA-2015-7-00332957.npz
4013
LOADED
dhs_train/IA-2015-7-00332964.npz
4014
LOADED
dhs_train/IA-2015-7-00332974.npz
4015
LOADED
dhs_train/IA-2015-7-00332979.npz
4016
LOADED
dhs_train/IA-2015-7-00332989.npz
4017
LOADED
dhs_train/IA-2015-7-00333008.npz
4018
LOADED
dhs_train/IA-2015-7-00333040.npz
4019
LOADED
dhs_train/IA-2015-7-00333044.npz
4020
LOADED
dhs_train/IA-2015-7-00333047.npz
4021
LOADED
dhs_train/IA-2015-7-00333050.npz
4022
LOADED
dhs_train/

dhs_train/IA-2015-7-00334605.npz
4184
LOADED
dhs_train/IA-2015-7-00334612.npz
4185
LOADED
dhs_train/IA-2015-7-00334618.npz
4186
LOADED
dhs_train/IA-2015-7-00334628.npz
4187
LOADED
dhs_train/IA-2015-7-00334630.npz
4188
LOADED
dhs_train/IA-2015-7-00334641.npz
4189
LOADED
dhs_train/IA-2015-7-00334660.npz
4190
LOADED
dhs_train/IA-2015-7-00334665.npz
4191
LOADED
dhs_train/IA-2015-7-00334677.npz
4192
LOADED
dhs_train/IA-2015-7-00334679.npz
4193
LOADED
dhs_train/IA-2015-7-00334680.npz
4194
LOADED
dhs_train/IA-2015-7-00334693.npz
4195
LOADED
dhs_train/IA-2015-7-00334697.npz
4196
LOADED
dhs_train/IA-2015-7-00334716.npz
4197
LOADED
dhs_train/IA-2015-7-00334719.npz
4198
LOADED
dhs_train/IA-2015-7-00334738.npz
4199
LOADED
dhs_train/IA-2015-7-00334757.npz
4200
LOADED
dhs_train/IA-2015-7-00334777.npz
4201
LOADED
dhs_train/IA-2015-7-00334784.npz
4202
LOADED
dhs_train/IA-2015-7-00334788.npz
4203
LOADED
dhs_train/IA-2015-7-00334804.npz
4204
LOADED
dhs_train/IA-2015-7-00334809.npz
4205
LOADED
dhs_train/

dhs_train/IA-2015-7-00340097.npz
4367
LOADED
dhs_train/IA-2015-7-00340101.npz
4368
LOADED
dhs_train/IA-2015-7-00340104.npz
4369
LOADED
dhs_train/IA-2015-7-00340135.npz
4370
LOADED
dhs_train/IA-2015-7-00340137.npz
4371
LOADED
dhs_train/IA-2015-7-00340153.npz
4372
LOADED
dhs_train/IA-2015-7-00340156.npz
4373
LOADED
dhs_train/IA-2015-7-00340159.npz
4374
LOADED
dhs_train/IA-2015-7-00340165.npz
4375
LOADED
dhs_train/IA-2015-7-00340183.npz
4376
LOADED
dhs_train/IA-2015-7-00340188.npz
4377
LOADED
dhs_train/IA-2015-7-00340189.npz
4378
LOADED
dhs_train/IA-2015-7-00340201.npz
4379
LOADED
dhs_train/IA-2015-7-00340213.npz
4380
LOADED
dhs_train/IA-2015-7-00340221.npz
4381
LOADED
dhs_train/IA-2015-7-00340223.npz
4382
LOADED
dhs_train/IA-2015-7-00340229.npz
4383
LOADED
dhs_train/IA-2015-7-00340231.npz
4384
LOADED
dhs_train/IA-2015-7-00340255.npz
4385
LOADED
dhs_train/IA-2015-7-00340259.npz
4386
LOADED
dhs_train/IA-2015-7-00340263.npz
4387
LOADED
dhs_train/IA-2015-7-00340268.npz
4388
LOADED
dhs_train/

dhs_train/IA-2015-7-00350437.npz
4550
LOADED
dhs_train/IA-2015-7-00350441.npz
4551
LOADED
dhs_train/IA-2015-7-00350449.npz
4552
LOADED
dhs_train/IA-2015-7-00350452.npz
4553
LOADED
dhs_train/IA-2015-7-00350455.npz
4554
LOADED
dhs_train/IA-2015-7-00350463.npz
4555
LOADED
dhs_train/IA-2015-7-00350471.npz
4556
LOADED
dhs_train/IA-2015-7-00350483.npz
4557
LOADED
dhs_train/IA-2015-7-00350489.npz
4558
LOADED
dhs_train/IA-2015-7-00350492.npz
4559
LOADED
dhs_train/IA-2015-7-00350493.npz
4560
LOADED
dhs_train/IA-2015-7-00350498.npz
4561
LOADED
dhs_train/IA-2015-7-00350516.npz
4562
LOADED
dhs_train/IA-2015-7-00350539.npz
4563
LOADED
dhs_train/IA-2015-7-00350550.npz
4564
LOADED
dhs_train/IA-2015-7-00350555.npz
4565
LOADED
dhs_train/IA-2015-7-00350556.npz
4566
LOADED
dhs_train/IA-2015-7-00350562.npz
4567
LOADED
dhs_train/IA-2015-7-00350568.npz
4568
LOADED
dhs_train/IA-2015-7-00350570.npz
4569
LOADED
dhs_train/IA-2015-7-00350574.npz
4570
LOADED
dhs_train/IA-2015-7-00350581.npz
4571
LOADED
dhs_train/

In [21]:
new_v

,index,NDVI,AWEI,NDWI,MNDWI,BLUE,GREEN,RED,NIR,SWIR_1,...,lat,lon,n_asset,asset_index,n_water,water_index,cluster_id,adm1dhs,urban,path
0,0,0.110300,-0.333849,-0.050746,-0.337540,0.558278,0.589286,0.632181,0.403698,0.341378,...,21.595282,74.213171,19,1.976667,19,3.631579,201427,20,R,dhs_train/IA-2015-7-00201427.npz
1,1,0.121075,-0.978598,-0.138141,-0.376654,0.341627,0.328375,0.259338,0.090657,0.072596,...,20.684193,78.067572,20,0.723061,20,3.900000,201441,20,R,dhs_train/IA-2015-7-00201441.npz
2,2,0.031069,-0.203146,-0.005858,-0.324705,0.397065,0.469174,0.397621,0.193908,0.132136,...,24.766001,93.898605,21,1.428244,21,3.952381,210007,21,R,dhs_train/IA-2015-7-00210007.npz
3,3,0.215059,-0.039584,-0.027489,-0.336422,0.592165,0.641385,0.485860,0.304202,0.289477,...,25.001055,93.881067,22,0.955851,22,3.454545,210043,21,R,dhs_train/IA-2015-7-00210043.npz
4,4,0.099453,-0.718598,-0.107267,-0.324674,0.231539,0.243249,0.201918,0.084458,0.054491,...,24.484923,93.772076,21,0.242614,21,3.571429,210045,21,U,dhs_train/IA-2015-7-00210045.npz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,2803,0.193601,-0.889771,-0.264392,-0.442002,0.206246,0.248335,0.223861,0.107378,0.084004,...,16.467705,75.244017,20,1.398734,20,4.700000,161116,16,R,dhs_train/IA-2015-7-00161116.npz
2804,2804,-0.075525,0.156048,0.193609,0.004409,0.341498,0.321984,0.200534,0.048180,0.034025,...,13.616641,74.694511,20,2.506196,20,4.750000,161119,16,U,dhs_train/IA-2015-7-00161119.npz
2805,2805,0.122449,-0.809472,-0.123800,-0.403351,0.425081,0.466380,0.393401,0.163687,0.134043,...,13.156055,78.394542,20,1.624081,20,4.600000,161140,16,U,dhs_train/IA-2015-7-00161140.npz
2806,2806,0.261325,-0.059723,0.012321,-0.419575,0.311093,0.368118,0.271661,0.169611,0.117677,...,12.710740,75.158273,22,1.365633,22,4.409091,161147,16,R,dhs_train/IA-2015-7-00161147.npz


In [99]:
file_0 = './new_data/train_chunk_0_v2.csv'
file_1 = './new_data/train_chunk_1_v2.csv'
file_2 = './new_data/train_chunk_2_v2.csv'
file_3 = './new_data/train_chunk_3_v2.csv'
file_4 = './new_data/train_chunk_4_v2.csv'

df0 = pd.read_csv(file_0)
df1 = pd.read_csv(file_1)
df2 = pd.read_csv(file_2)
df3 = pd.read_csv(file_3)
df4 = pd.read_csv(file_4)

In [100]:
df_total = pd.concat((df0, df1, df2, df3, df4), ignore_index = True)

In [101]:
x_cols = ['NDVI', 'AWEI', 'NDWI', 'MNDWI', 'BLUE',
       'GREEN', 'RED', 'NIR', 'SWIR_1', 'THERMAL', 'SWIR_2', 'PANCH',
        'lat', 'lon',
        'n_asset', 'asset_index']

for c in x_cols:
    df_total[c] = df_total[c].replace(np.NaN, df_total[c].median())


In [93]:
df_total.to_csv('train_final_v2.csv')

In [102]:
from sklearn import linear_model

In [103]:
df_total.columns

Index(['Unnamed: 0.2', 'index', 'NDVI', 'AWEI', 'NDWI', 'MNDWI', 'BLUE',
       'GREEN', 'RED', 'NIR', 'SWIR_1', 'THERMAL', 'SWIR_2', 'PANCH',
       'Unnamed: 0.1', 'Unnamed: 0', 'DHSID_EA', 'cname', 'year', 'lat', 'lon',
       'n_asset', 'asset_index', 'n_water', 'water_index', 'cluster_id',
       'adm1dhs', 'urban', 'path'],
      dtype='object')

In [104]:
X = df_total[x_cols]
y = df_total['water_index']

In [105]:
regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression()

In [94]:
df_masked = pd.read_csv('test_masked_v2.csv')

In [97]:
df_masked
for c in x_cols:
    df_masked[c] = df_masked[c].replace(np.NaN, df_masked[c].median())


In [106]:
X_masked = df_masked[x_cols]

In [107]:
regr.predict(X_masked)

array([4.65838491, 4.82375314, 3.93652927, ..., 4.94857951, 4.09161678,
       4.21447994])

In [108]:
new_df = pd.DataFrame()
new_df['DHSID_EA'] = df_masked['DHSID_EA']
new_df['water_index'] = regr.predict(X_masked)

In [110]:
new_df.to_csv('submission_2.csv')